In [1]:
# permission for the json to act
! chmod 600 /content/kaggle.json

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d dqmonn/zalando-store-crawl

100% 2.24G/2.24G [00:34<00:00, 106MB/s]
100% 2.24G/2.24G [00:34<00:00, 69.9MB/s]


In [5]:
# Mengekstrak file dataset

import zipfile
zip_file = zipfile.ZipFile('/content/zalando-store-crawl.zip')
zip_file.extractall('/tmp/')

In [6]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# Split folders
import splitfolders
from sklearn.model_selection import train_test_split

input_folder = '/tmp/zalando'

# train, val, test
splitfolders.ratio(input_folder, output="/tmp/zalando2", 
                   seed=42, ratio=(.7, .2, .1), 
                   group_prefix=None) # default values

Copying files: 16186 files [00:05, 2989.63 files/s]


In [8]:
!rm -rf /tmp/zalando2/test/zalando

In [9]:
!rm -rf /tmp/zalando2/train/zalando

In [10]:
!rm -rf /tmp/zalando2/val/zalando

In [11]:
# Import ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
# Training generator initialization

train_generator = ImageDataGenerator(rescale=1/255,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     rotation_range=60,
                                     zoom_range=0.3,
                                     fill_mode='nearest')

In [13]:
# Training data generator (batch_size=1200) (class_mode='categorical')
train_data = train_generator.flow_from_directory('/tmp/zalando2/train',
                                                 target_size=(224, 224),
                                                 batch_size=120,
                                                 class_mode='categorical',)

Found 11326 images belonging to 6 classes.


In [14]:
# Validation data generator (batch_size=30) (class_mode='categorical')

val_generator = ImageDataGenerator(rescale=1/255)
val_data = val_generator = val_generator.flow_from_directory('/tmp/zalando2/val',
                                                             target_size=(224, 224),
                                                             class_mode='categorical',
                                                             batch_size=33)

Found 3235 images belonging to 6 classes.


In [15]:
# Construct 8-layer model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential ([
    Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(3,3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax'),
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 128)               5

In [17]:
# Compile model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [1]:
# Training model (See step per epoch)

history = model.fit(train_data,
                    steps_per_epoch=100/1,
                    epochs=10,
                    validation_data=val_data,
                    validation_steps=100/1, 
                    verbose=2)

NameError: ignored

In [ ]:
# plot perubahan nilai loss dan akurasi pada tiap epoch saat training

import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Nilai Loss pada tiap epoch')